In [ ]:
import pandas as pd

from src.data.models import Dataset

In [ ]:
legislatures = Dataset(
    name="legislatures",
    awde_endpoint="parliament-periods",
    awde_params={
        "label": "Bundestag 2021 - 2025",
        "type": "legislature",
    },
)
legislatures.fetch()

In [ ]:
legislatures.data

In [ ]:
# Abstimmungen im aktuellen Bundestag:
polls = Dataset(
    name="polls",
    awde_endpoint="polls",
    awde_params={"field_legislature[entity.id]": 132}
)

def transform_polls(data: pd.DataFrame) -> pd.DataFrame:
    df = data.copy()
    df.rename(
        {
            "field_accepted": "accepted",
            "field_poll_date": "date"
        },
        axis=1,
        inplace=True
    )
    df["fid_legislatur"] = df.field_legislature.apply(lambda x: x["id"])
    df["fid_topic"] = df.field_topics.apply(lambda x: ",".join( [str(i["id"]) for i in x] ) if x is not None else None)
    df = df[["id", "fid_legislatur", "fid_topic", "label", "date", "accepted"]]

    return df

polls.transform_data = transform_polls

In [ ]:
polls.data

In [ ]:
poll_ids = polls.data.id

all_votes = {}

for id in poll_ids:
    all_votes[id] = Dataset(name=f"poll_{id}", awde_endpoint="votes", awde_params={"poll": id})
    if all_votes[id].data is None:
        print(id, end=" ")
        all_votes[id].fetch()
        all_votes[id].save()

In [ ]:
def transform_vote(data):
    df = data.copy()

    df["name"] = df.mandate.apply(lambda d: d["label"].split(" (Bundestag")[0] if d is not None else None)
    df["fid_vote"] = df.id
    df["fid_poll"] = df.poll.apply(lambda d: d["id"])
    df["fraction"] = df.fraction.apply(lambda d: d["label"].split(" (Bundestag")[0] if d is not None else None)


    df = df[[
        "fid_poll",
        "fid_vote",
        "name",
        "fraction",
        "vote",
        "reason_no_show",
        "reason_no_show_other",
    ]]

    return df

for k, v in all_votes.items():
    v.transform_data = transform_vote

allvotes = pd.concat([x.data for x in all_votes.values()])

In [ ]:
allvotes

In [ ]:
polls.data

In [ ]:
df = (
    allvotes
    .set_index("fid_poll")
    .join(polls.data.set_index("id"))
    .reset_index()
    .rename({"fid_poll": "poll_id", "fid_vote": "vote_id"}, axis=1)
)
    # .set_index(["poll_id", "fid_topic", "fid_legislatur", "label", "date", "accepted"])

df.date = pd.to_datetime(df.date)

Hier die Fraktionen:

In [ ]:
df.fraction.value_counts()

Wir vereinfachen die Fraktionsnamen für das weitere Arbeiten:

In [ ]:
df.fraction = df.fraction.replace({
    "SPD": "SPD",
    "CDU/CSU": "CDU",
    "BÜNDNIS 90/DIE GRÜNEN": "GRÜ",
    "FDP": "FDP",
    "AfD": "AfD",
    "DIE LINKE.": "LIN",
    "Die Linke. (Gruppe)": "LIN",
    "BSW (Gruppe)": "BSW",
})

Der Abstimmungs-Datensatz enthält bereits das Attribut "accepted" (*angenommen*, ja/nein).
Wir fügen noch aus den Stimm-Daten hinzu, wie das Stimmenverhältnis war.

In [ ]:
p = df.groupby("poll_id").aggregate(
    quorum=("poll_id", "size"),
    sum_yes=("vote", lambda x: (x == "yes").sum()),
    sum_no=("vote", lambda x: (x == "no").sum()),
    sum_nil=("vote", lambda x: x.isin(["abstain", "no_show"]).sum()),
)

In [ ]:
df = df.set_index("poll_id").join(p)

Es folgen die Stimmenzahlen aus den Fraktionen. Achtung: Abgeordnete der Linken sind zwischen dem 6. Dezember 2023 und 2. Februar 2024 als fraktionslos aufgeführt, danach entweder als "Die Linke (Gruppe)" oder "BSW (Gruppe)". Für die Frage nach dem Abstimmverhalten im Verhältnis zur eigenen Fraktion nehmen wir daher diese Phase aus dem Datensatz heraus. Das gilt dann auch für alle weiteren fraktionslosen.

In [ ]:
old_nrow = len(df)

df = df.loc[df.fraction.ne("fraktionslos")]

(old_nrow - len(df), (old_nrow - len(df)) / old_nrow * 100)

Wir verlieren dadurch 1145 Stimmen oder ca. 1%. Das ist zu verschmerzen.

In [ ]:
df.fraction.value_counts()

In [ ]:
p = df.groupby(["poll_id", "fraction"]).aggregate(
    yes=("vote", lambda x: (x == "yes").sum()),
    no=("vote", lambda x: (x == "no").sum()),
    nil=("vote", lambda x: x.isin(["abstain", "no_show"]).sum()),
)
p = p.reset_index().pivot(index="poll_id", columns="fraction").reorder_levels([1,0], axis=1)

p.columns = ["_".join(col) for col in p.columns]

p = p.astype(pd.Int64Dtype())
p

In [ ]:
df = df.join(p)

In [ ]:
df